# Implementacja Eksperckiego Systemu dla Zwierząt

Przykład z [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

W tym przykładzie zaimplementujemy prosty system oparty na wiedzy, który pozwala określić zwierzę na podstawie jego cech fizycznych. System można przedstawić za pomocą następującego drzewa AND-OR (to tylko część całego drzewa, łatwo możemy dodać więcej reguł):

![](../../../../lessons/2-Symbolic/images/AND-OR-Tree.png)


## Nasza własna powłoka systemów eksperckich z wnioskowaniem wstecznym

Spróbujmy zdefiniować prosty język do reprezentacji wiedzy oparty na regułach produkcji. Użyjemy klas Pythona jako słów kluczowych do definiowania reguł. Będą zasadniczo 3 typy klas:
* `Ask` reprezentuje pytanie, które należy zadać użytkownikowi. Zawiera zestaw możliwych odpowiedzi.
* `If` reprezentuje regułę i jest jedynie syntaktycznym ułatwieniem do przechowywania treści reguły.
* `AND`/`OR` to klasy reprezentujące gałęzie AND/OR drzewa. Przechowują jedynie listę argumentów w środku. Aby uprościć kod, cała funkcjonalność jest zdefiniowana w klasie nadrzędnej `Content`.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

W naszym systemie pamięć robocza zawierałaby listę **faktów** jako **pary atrybut-wartość**. Bazę wiedzy można zdefiniować jako jeden duży słownik, który mapuje działania (nowe fakty, które powinny zostać wprowadzone do pamięci roboczej) na warunki, wyrażone jako wyrażenia AND-OR. Ponadto niektóre fakty mogą być `Zapytane`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Aby przeprowadzić wnioskowanie wsteczne, zdefiniujemy klasę `Knowledgebase`. Będzie ona zawierać:
* `Pamięć roboczą` - słownik, który mapuje atrybuty na wartości
* `Zasady bazy wiedzy` w formacie zdefiniowanym powyżej

Dwa główne metody to:
* `get`, aby uzyskać wartość atrybutu, wykonując w razie potrzeby wnioskowanie. Na przykład, `get('color')` pobierze wartość dla slotu koloru (jeśli zajdzie potrzeba, zapyta o wartość i zapisze ją do późniejszego użycia w pamięci roboczej). Jeśli zapytamy `get('color:blue')`, zapyta o kolor, a następnie zwróci wartość `y`/`n` w zależności od koloru.
* `eval` wykonuje rzeczywiste wnioskowanie, czyli przechodzi przez drzewo AND/OR, ocenia podcele, itd.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Teraz zdefiniujmy naszą bazę wiedzy o zwierzętach i przeprowadźmy konsultację. Zauważ, że to wywołanie będzie zadawać Ci pytania. Możesz odpowiadać, wpisując `y`/`n` na pytania tak/nie lub podając liczbę (0..N) na pytania z dłuższymi odpowiedziami wielokrotnego wyboru.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Korzystanie z PyKnow do wnioskowania w przód

W poniższym przykładzie spróbujemy zaimplementować wnioskowanie w przód, korzystając z jednej z bibliotek do reprezentacji wiedzy, [PyKnow](https://github.com/buguroo/pyknow/). **PyKnow** to biblioteka do tworzenia systemów wnioskowania w przód w Pythonie, zaprojektowana w sposób przypominający klasyczny, starszy system [CLIPS](http://www.clipsrules.net/index.html).

Moglibyśmy również zaimplementować wnioskowanie w przód samodzielnie bez większych trudności, ale naiwne implementacje zazwyczaj nie są zbyt wydajne. Do bardziej efektywnego dopasowywania reguł używa się specjalnego algorytmu [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

Zdefiniujemy nasz system jako klasę, która dziedziczy po `KnowledgeEngine`. Każda reguła jest definiowana przez osobną funkcję z adnotacją `@Rule`, która określa, kiedy reguła powinna zostać uruchomiona. Wewnątrz reguły możemy dodawać nowe fakty za pomocą funkcji `declare`, a dodanie tych faktów spowoduje wywołanie kolejnych reguł przez silnik wnioskowania w przód.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Gdy zdefiniujemy bazę wiedzy, wypełniamy naszą pamięć roboczą początkowymi faktami, a następnie wywołujemy metodę `run()`, aby przeprowadzić wnioskowanie. W rezultacie można zauważyć, że nowe wywnioskowane fakty są dodawane do pamięci roboczej, w tym ostateczny fakt dotyczący zwierzęcia (jeśli poprawnie ustawimy wszystkie początkowe fakty).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby zapewnić poprawność tłumaczenia, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
